In [1]:
!pip install flask requests transformers

In [2]:
%%writefile app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

# Pre-made responses
responses = {
    "hello": "Hi there! How can I help you today?",
    "bye": "Goodbye! Have a nice day.",
    "default": "I'm not sure how to respond to that."
}

# Initialize sentiment analysis pipelines for two different models
sentiment_pipeline_distil = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
sentiment_pipeline_nlptown = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def analyze_sentiment(message, model="distil"):
    """
    Analyze the sentiment of the message using the selected model.

    Args:
        message (str): The text to analyze.
        model (str): 'distil' or 'nlptown'

    Returns:
        dict: A dictionary with the sentiment analysis result.
    """
    if model == "distil":
        # Returns a list like [{'label': 'POSITIVE', 'score': 0.99}]
        result = sentiment_pipeline_distil(message)
        return result[0]
    elif model == "nlptown":
        # Returns a list like [{'label': '4 stars', 'score': 0.75}]
        result = sentiment_pipeline_nlptown(message)
        label = result[0]['label']
        # Extract numeric rating from the label (e.g. "2 stars" -> 2)
        try:
            stars = int(label.split()[0])
        except Exception:
            stars = 3  # default to neutral if parsing fails
        # Include the parsed stars for easier evaluation
        return {"label": label, "score": result[0]['score'], "stars": stars}
    else:
        return {"label": "UNKNOWN", "score": 0.0}

@app.route('/chat', methods=['POST'])
def chat():
    data = request.get_json()
    user_message = data.get("message", "")
    model_choice = data.get("model", "distil")  # default to using the distil model

    sentiment_result = analyze_sentiment(user_message, model=model_choice)

    # Decide if the sentiment is negative (i.e., angry)
    angry = False
    if model_choice == "distil":
        if sentiment_result['label'] == "NEGATIVE":
            angry = True
    elif model_choice == "nlptown":
        # For the nlptown model, consider 1 or 2 stars as negative sentiment
        if sentiment_result.get("stars", 3) <= 2:
            angry = True

    if angry:
        reply = "I sense some anger in your words. Let's try to keep the conversation calm."
    else:
        # Use basic rule-based matching for greetings and goodbyes
        if "hello" in user_message.lower():
            reply = responses.get("hello")
        elif "bye" in user_message.lower():
            reply = responses.get("bye")
        else:
            reply = responses.get("default")

    return jsonify({
        "reply": reply,
        "sentiment_analysis": sentiment_result
    })

if __name__ == '__main__':
    # The server will run on 0.0.0.0:5000
    app.run(host='0.0.0.0', port=5000)


Overwriting app.py


In [3]:
!nohup python app.py &


nohup: appending output to 'nohup.out'


In [4]:
import socket
import time

def check_port(host, port):
  port_closed = True
  while port_closed:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        try:
            sock.connect((host, port))
            print(f"Port {port} on {host} is open!")
            port_closed = False
        except ConnectionRefusedError:
            print(f"Port {port} on {host} is closed.")
            time.sleep(0.5)


check_port("127.0.0.1", 5000)

Port 5000 on 127.0.0.1 is open!


In [5]:
import requests

url = "http://127.0.0.1:5000/chat"

# Test messages along with a choice of sentiment model.
# Use "distil" for the distilbert model and "nlptown" for the nlptown model.
test_messages = [
    {"message": "Hello there!", "model": "distil"},
    {"message": "I hate this so much!", "model": "distil"},
    {"message": "This is absolutely terrible!", "model": "nlptown"},
    {"message": "Goodbye", "model": "nlptown"}
]

for payload in test_messages:
    response = requests.post(url, json=payload)
    if response.ok:
        result = response.json()
        print(f"User: {payload['message']} (Model: {payload['model']})")
        print(f"Chatbot: {result['reply']}")
        print(f"Sentiment: {result['sentiment_analysis']}")
        print("-" * 50)
    else:
        print("Failed to connect to the chatbot service.")


User: Hello there! (Model: distil)
Chatbot: Hi there! How can I help you today?
Sentiment: {'label': 'POSITIVE', 'score': 0.9994440674781799}
--------------------------------------------------
User: I hate this so much! (Model: distil)
Chatbot: I sense some anger in your words. Let's try to keep the conversation calm.
Sentiment: {'label': 'NEGATIVE', 'score': 0.9994558691978455}
--------------------------------------------------
User: This is absolutely terrible! (Model: nlptown)
Chatbot: I sense some anger in your words. Let's try to keep the conversation calm.
Sentiment: {'label': '1 star', 'score': 0.9742461442947388, 'stars': 1}
--------------------------------------------------
User: Goodbye (Model: nlptown)
Chatbot: Goodbye! Have a nice day.
Sentiment: {'label': '5 stars', 'score': 0.5722659826278687, 'stars': 5}
--------------------------------------------------
